In [1]:
using CompTime, InteractiveUtils, MacroTools

In [2]:
cleanup = MacroTools.flatten ∘ Base.remove_linenums!

MacroTools.flatten ∘ Base.remove_linenums!

In [3]:
function pw_rec(n, x)
    if iszero(n)
        one(x)
    elseif isone(n)
        x
    elseif isodd(n)
        x * pw_rec(n - 1, x)
    else
        pw_rec(n ÷ 2, x * x)
    end
end

pw_rec (generic function with 1 method)

In [4]:
Tuple(pw_rec(n, "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [5]:
function pw_rec_pe(::Val{n}, x) where {n}
    if iszero(n)
        one(x)
    elseif isone(n)
        x
    elseif isodd(n)
        x * pw_rec_pe(Val(n - 1), x)
    else
        pw_rec_pe(Val(n ÷ 2), x * x)
    end
end

pw_rec_pe (generic function with 1 method)

In [6]:
Tuple(pw_rec_pe(Val(n), "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [7]:
@code_typed pw_rec_pe(Val(5), 10)

CodeInfo(
1 ─      nothing::Nothing
│        nothing::Nothing
│   %3 = Base.mul_int(x, x)::Int64
│   %4 = Base.mul_int(%3, %3)::Int64
│   %5 = Base.mul_int(x, %4)::Int64
└──      return %5
) => Int64

In [8]:
function pw_tail_rec(n, x)
    if iszero(n)
        one(x)
    else
        pw_tail_rec(n, one(x), x)
    end
end

pw_tail_rec (generic function with 1 method)

In [9]:
function pw_tail_rec(n, r, x)
    @assert !iszero(n)
    if isone(n)
        r * x
    elseif isodd(n)
        pw_tail_rec(n - 1, r * x, x)
    else
        pw_tail_rec(n ÷ 2, r, x * x)
    end
end

pw_tail_rec (generic function with 2 methods)

In [10]:
Tuple(pw_tail_rec(n, "Abc") for n = 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [11]:
function pw_loop(n, x)
    if iszero(n)
        return one(x)
    else
        r = one(x)

        while n > 1
            if isodd(n)
                r = r * x
                n = n - 1
            else
                x = x * x
                n = n ÷ 2
            end
        end

        return r * x
    end
end

pw_loop (generic function with 1 method)

In [12]:
Tuple(pw_loop(n, "Abc") for n = 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [13]:
@generated function pw_gen(::Val{n}, x) where {n}
    b = Expr[]

    if iszero(n)
        push!(b, :(return one(x)))
    else
        push!(b, :(r = one(x)))
        k = n

        while k > 1
            if isodd(k)
                push!(b, :(r = r * x))
                k = k - 1
            else
                push!(b, :(x = x * x))
                k = k ÷ 2
            end
        end

        push!(b, :(return r * x))
        
    end
    
    b = quote $(b...) end
    Core.println(string(cleanup(b)))
    return b
end

pw_gen (generic function with 1 method)

In [14]:
pw_gen(Val(7), "Abc")

begin
    r = one(x)
    r = r * x
    x = x * x
    r = r * x
    x = x * x
    return r * x
end


"AbcAbcAbcAbcAbcAbcAbc"

In [15]:
@ct_enable function pw_ct(@ct(n), x)
    @ct_ctrl if iszero(n)
        return one(x)
    else
        r = one(x)
        @ct(k = n)

        @ct_ctrl while k > 1
            @ct_ctrl if isodd(k)
                r = r * x
                @ct(k = k - 1)
            else
                x = x * x
                @ct(k = k ÷ 2)
            end
        end

        return r * x
    end
end

runtime (generic function with 1 method)

In [16]:
Tuple(pw_ct(Val{n}, "Abc") for n = 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [17]:
Tuple(runtime(pw_ct, n, "Abc") for n = 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [18]:
Tuple(comptime(pw_ct, Val{n}, "Abc") for n = 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [19]:
@code_typed pw_ct(Val{5}, 10)

CodeInfo(
1 ─ %1 = Base.mul_int(1, x)::Int64
│   %2 = Base.mul_int(x, x)::Int64
│   %3 = Base.mul_int(%2, %2)::Int64
│   %4 = Base.mul_int(%1, %3)::Int64
└──      return %4
) => Int64

In [20]:
debug(pw_ct, Val{5}, 10) |> cleanup

quote
    r = one(x)
    5
    r = r * x
    4
    x = x * x
    2
    x = x * x
    1
    return r * x
end

In [21]:
@ct_enable function pw_opt_ct(@ct(n), x)
    @ct_ctrl if iszero(n)
        return one(x)
    else
        @ct(r1 = true)
        @ct(k = n)

        @ct_ctrl while k > 1
            @ct_ctrl if isodd(k)
                @ct_ctrl if r1
                    r = x
                    @ct(r1 = false)
                else
                    r = r * x
                end
                @ct(k = k - 1)
            else
                x = x * x
                @ct(k = k ÷ 2)
            end
        end

        @ct_ctrl if r1
            return x
        else
            return r * x
        end
    end
end

runtime (generic function with 2 methods)

In [22]:
Tuple(pw_opt_ct(Val{n}, "Abc") for n in 0:5)

("", "Abc", "AbcAbc", "AbcAbcAbc", "AbcAbcAbcAbc", "AbcAbcAbcAbcAbc")

In [23]:
@code_typed pw_opt_ct(Val{5}, 10)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
│   %2 = Base.mul_int(%1, %1)::Int64
│   %3 = Base.mul_int(x, %2)::Int64
└──      return %3
) => Int64

In [24]:
debug(pw_opt_ct, Val{0}, 10) |> cleanup

:(return one(x))

In [25]:
debug(pw_ct, Val{1}, 10) |> cleanup

quote
    r = one(x)
    1
    return r * x
end

In [26]:
debug(pw_opt_ct, Val{4}, 10) |> cleanup

quote
    true
    4
    x = x * x
    2
    x = x * x
    1
    return x
end

In [27]:
debug(pw_opt_ct, Val{5}, 10) |> cleanup

quote
    true
    5
    r = x
    false
    4
    x = x * x
    2
    x = x * x
    1
    return r * x
end

In [28]:
debug(pw_opt_ct, Val{7}, 10) |> cleanup

quote
    true
    7
    r = x
    false
    6
    x = x * x
    3
    r = r * x
    2
    x = x * x
    1
    return r * x
end